<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/query_engine/sec_tables/tesla_10q_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Joint Tabular/Semantic QA over Tesla 10K

In this example, we show how to ask questions over 10K with understanding of both the unstructured text as well as embedded tables.

We use Unstructured to parse out the tables, and use LlamaIndex recursive retrieval to index/retrieve tables if necessary given the user question.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [45]:
!pip install llama-index

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

## Perform Data Extraction

In these sections we use Unstructured to parse out the table and non-table elements.

### Extract Elements

We use Unstructured to extract table and non-table elements from the 10-K filing.

In [ ]:
!wget "https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1" -O tesla_2021_10k.htm
!wget "https://www.dropbox.com/scl/fi/rkw0u959yb4w8vlzz76sa/tesla_2020_10k.htm?rlkey=tfkdshswpoupav5tqigwz1mp7&dl=1" -O tesla_2020_10k.htm

In [ ]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

reader = FlatReader()
docs_2021 = reader.load_data(Path("tesla_2021_10k.htm"))
docs_2020 = reader.load_data(Path("tesla_2020_10k.htm"))

In [41]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser()

In [42]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

In [43]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [49]:
[isinstance(b,IndexNode) for b in base_nodes_2021]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [47]:
example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][
    20
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
)

IndexError: ignored

## Setup Recursive Retriever

Now that we've extracted tables and their summaries, we can setup a recursive retriever in LlamaIndex to query these tables.

### Construct Retrievers

In [50]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex

In [51]:
# construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

In [55]:
import os
import pickle

if not os.path.exists("2021_nodes.pkl"):
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))
else:
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

In [60]:
from llama_index.node_parser import (
    UnstructuredElementNodeParser,
)

node_parser = UnstructuredElementNodeParser()

In [64]:
raw_nodes_2021

[TextNode(id_='98874c24-f88a-487c-bcee-bc6339393f78', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f407a99b-e878-45c8-88b2-ae8ba162e09b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ad006e17c8f5260fb254c3cd492f1b429a982dcf70498f3438bcfe22b76c6b1e'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c42dc9fe-ad77-4782-b637-b5470ac793de', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='6aaa3cfa59265e9ce9eba45cf76121c3013fee6e5e76a42c0117f280d45d097d')}, hash='33c2a90c62d860c4e8e0df55537a5f4429789ff6dd0c413c1a56c573c8e59b37', text='UNITED STATES\n\nSECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\nFORM 10-K\n\n(Mark One)\n\n☒ \n     ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the fiscal year ended December 31, 2021\n\nOR\n\n☐ \n     TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF TH

In [61]:
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
    raw_nodes_2021
)

In [73]:
vector_index = VectorStoreIndex(base_nodes_2021)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

In [74]:
example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][
    20
]

# Index Node
print(
    f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
)
# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")
# Referenceed Table
print(
    f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
)

IndexError: ignored

In [67]:
!pip install llama_index[langchain]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 6.3 MB/s eta 0:00:00


In [68]:
from llama_index.langchain_helpers.text_splitter import SentenceSplitter

In [105]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector", #SentenceSplitter,
    retriever_dict={"vector": vector_retriever},
    node_dict=node_mappings_2021,
    verbose=False,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [106]:
nodes = query_engine.retrieve("What was the revenue for Tesla in 2020?")


In [81]:
nodes

[NodeWithScore(node=TextNode(id_='6c6d3551-08d1-477d-abdd-54c6073e786f', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f407a99b-e878-45c8-88b2-ae8ba162e09b', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ad006e17c8f5260fb254c3cd492f1b429a982dcf70498f3438bcfe22b76c6b1e'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='41da8a8a-44e0-4d01-8245-619731f195ae', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='42daa59110c56ea0f8abc37061f8ba7f94059f9e40eb56078ebabda4fee53bf6'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='cf7d31cb-3e3e-454d-a0b3-a3c03981f3a9', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d6f4f5bb390d2812ef5692445f6ff28015f9fd6934c9b4a6138691c401f3696e')}, hash='797c1fcf4d82afadea506cf1a95af0a13bb2da33dec8ca6370da753da13f4668', text='Revenue from the sale of automotive regulatory credits totaled $\n\n1.46\n\nbillion,

In [ ]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)


### Run some Queries

In [111]:
response = query_engine.query("What was the R&D in 2020?")
print(str(response))

The R&D expenses in 2020 were $1.10 billion.


In [ ]:
# compare against the baseline retriever
response = vector_query_engine.query("What was the revenue in 2020?")
print(str(response))

The revenue in 2020 was a number.


In [114]:
response = query_engine.query("What were the total cash flows in 2021 and in 2020?")

In [115]:
print(str(response))

The total cash flows in 2021 were $5.20 billion, while the total cash flows in 2020 were $9.97 billion.


In [89]:
response = vector_query_engine.query("What were the total cash flows in 2020?")
print(str(response))

The total cash flows in 2020 were $9.97 billion net cash provided by financing activities.


In [118]:
response = query_engine.query("What are the financial risk factors (interest rate risk, foreign exhange risk and inflation risk) for Tesla? List in bullet points and also any hedges they have")
print(str(response))

- Interest rate risk: Tesla may be exposed to interest rate risk, as changes in interest rates can impact the cost of borrowing and financing activities. Fluctuations in interest rates could affect Tesla's ability to obtain favorable financing terms and could increase its interest expense.
- Foreign exchange risk: Tesla operates globally and generates revenue in various currencies. Fluctuations in foreign exchange rates can impact the translation of foreign currency-denominated revenues and expenses into the reporting currency (e.g., US dollars). This can result in gains or losses on foreign currency transactions and affect Tesla's financial results.
- Inflation risk: Inflation can erode the purchasing power of Tesla's cash flows and assets over time. Rising inflation rates can increase the cost of raw materials, labor, and other inputs, potentially impacting Tesla's profitability and financial performance.

Hedges:
The context information does not provide specific details about any he

In [91]:
response = vector_query_engine.query("What are the risk factors for Tesla?")
print(str(response))

The risk factors for Tesla include the need to ensure compliance with regulatory requirements in various jurisdictions, the dependence on consumer demand for electric vehicles, competition in the automotive industry, perceptions about the limited range and access to charging facilities for electric vehicles, volatility in the cost of oil and gasoline, government regulations and economic incentives, concerns about future viability, cyclical sales in the automotive industry, potential failures or challenges in the supply chain, including component shortages and the ability to secure alternative sources, and potential issues in increasing localized procurement at manufacturing facilities.


## Try Table Comparisons

In this setting we load in both the 2021 and 2020 10K filings, parse each into a hierarchy of tables/text objects, define a recursive retriever over each, and then compose both with a SubQuestionQueryEngine.

This allows us to execute document comparisons against both.

### Define E2E Recursive Retriever Function

In [92]:
import pickle
import os


def create_recursive_retriever_over_doc(docs, nodes_save_path=None):
    """Big function to go from document path -> recursive retriever."""
    node_parser = UnstructuredElementNodeParser()
    if nodes_save_path is not None and os.path.exists(nodes_save_path):
        raw_nodes = pickle.load(open(nodes_save_path, "rb"))
    else:
        raw_nodes = node_parser.get_nodes_from_documents(docs)
        if nodes_save_path is not None:
            pickle.dump(raw_nodes, open(nodes_save_path, "wb"))

    base_nodes, node_mappings = node_parser.get_base_nodes_and_mappings(
        raw_nodes
    )

    ### Construct Retrievers
    # construct top-level vector index + query engine
    vector_index = VectorStoreIndex(base_nodes)
    vector_retriever = vector_index.as_retriever(similarity_top_k=2)
    recursive_retriever = RecursiveRetriever(
        "vector",
        retriever_dict={"vector": vector_retriever},
        node_dict=node_mappings,
        verbose=True,
    )
    query_engine = RetrieverQueryEngine.from_args(recursive_retriever)
    return query_engine, base_nodes

### Create Sub Question Query Engine

In [93]:
import nest_asyncio

nest_asyncio.apply()

In [94]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [95]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-4")

service_context = ServiceContext.from_defaults(llm=llm)

In [96]:
query_engine_2021, nodes_2021 = create_recursive_retriever_over_doc(
    docs_2021, nodes_save_path="2021_nodes.pkl"
)
query_engine_2020, nodes_2020 = create_recursive_retriever_over_doc(
    docs_2020, nodes_save_path="2020_nodes.pkl"
)

100%|██████████| 7/7 [00:22<00:00,  3.15s/it]


In [97]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_2021,
        metadata=ToolMetadata(
            name="tesla_2021_10k",
            description=(
                "Provides information about Tesla financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=query_engine_2020,
        metadata=ToolMetadata(
            name="tesla_2020_10k",
            description=(
                "Provides information about Tesla financials for year 2020"
            ),
        ),
    ),
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

### Try out some Comparisons

In [98]:
response = sub_query_engine.query(
    "Can you compare and contrast the cash flow in 2021 with 2020?"
)

Generated 2 sub questions.
[tesla_2021_10k] Q: What was the cash flow of Tesla in 2021?
Retrieving with query id None: What was the cash flow of Tesla in 2021?
Retrieving text node: 189 
       
       
     $ 
     826 
       
       
     $ 
     31,015

The accompanying notes are an integral part of these consolidated financial statements.

52

Tesla, Inc.

Consolidated Statements of Cash Flows

(in millions)

Year Ended December 31, 
       
    
       
       
     2021 
       
       
     2020 
       
       
     2019 
       
    
     Cash Flows from Operating Activities 
       
       
       
       
       
       
       
       
       
    
     Net income (loss) 
       
     $ 
     5,644 
       
       
     $ 
     862 
       
       
     $ 
     ( 775 
     ) 
    
     Adjustments to reconcile net income (loss) to net cash provided by operating activities: 
       
       
       
       
       
       
       
       
       
    
     Depreciation, amor

In [99]:
print(str(response))

In 2021, Tesla had a cash flow of $5.20 billion, which was slightly lower than the $5.94 billion cash flow it had in 2020. This indicates a decrease in cash flow from 2020 to 2021.


In [100]:
response = sub_query_engine.query(
    "Can you compare and contrast the R&D expenditures in 2021 vs. 2020?"
)

Generated 2 sub questions.
[tesla_2021_10k] Q: What was the R&D expenditure for Tesla in 2021?
Retrieving with query id None: What was the R&D expenditure for Tesla in 2021?
Retrieving text node: 39

R&D expenses increased $1.10 billion, or 74%, in the year ended December 31, 2021 as compared to the year ended December 31, 2020. The increase was primarily due to a $506 million increase in employee and labor related expenses due to an increase in headcount, a $263 million increase in R&D expensed materials, a $211 million increase in facilities, outside services, freight and depreciation expense and an $103 million increase in stock-based compensation expense. These increases were to support our expanding product roadmap such as the new versions of Model S and Model X and technologies including our proprietary battery cells and there were additional R&D expenses as we were in the pre-production phases at both Gigafactory Texas and Gigafactory Berlin in the current year.

R&D expenses as

In [101]:
print(str(response))

In 2021, Tesla spent $2.593 billion on research and development (R&D), which is significantly higher than the $148 million they spent on R&D in 2020. This shows a substantial increase in R&D expenditure from 2020 to 2021.


In [102]:
response = sub_query_engine.query(
    "Can you compare and contrast the risk factors in 2021 vs. 2020?"
)

Generated 2 sub questions.
[tesla_2021_10k] Q: What were the risk factors for Tesla in 2021?
Retrieving with query id None: What were the risk factors for Tesla in 2021?
Retrieving text node: We also emphasize in our evaluation and career development efforts internal mobility opportunities for employees to drive professional development. Our goal is a long-term, upward-bound career at Tesla for every employee, which we believe also drives our retention efforts.

Our ability to retain our talented workforce is correlated to our compensation practices and culture of open communication. We provide a highly competitive wage that meets or exceeds that of comparable manufacturing roles, even before equity and benefits are factored in. In addition, the majority of our employees have the opportunity to receive additional Tesla equity each year based on their performance. We continue to review salary and wages against benchmarks and adjust to ensure wages are competitive, and have instituted a 

In [103]:
print(str(response))

In both 2020 and 2021, Tesla faced risk factors related to the global COVID-19 pandemic, which could disrupt operations, deliveries, and business activities. However, the specific challenges differed slightly between the two years. In 2020, Tesla's risks were more focused on logistical issues such as increasing delivery volumes, particularly in international markets, and ramping up logistics channels in China and Europe. They also faced challenges in increasing the number of Supercharger stations and connectors, and meeting sales, delivery, installation, servicing, and vehicle charging targets globally. 

In contrast, the 2021 risks were more centered around personnel and cybersecurity. Tesla's ability to attract and retain senior leadership and a large number of skilled personnel was a significant concern, especially in regions with strong competition. The potential departure of key employees and negative publicity were also seen as risks. Cybersecurity threats, including cyber-attack

#### Try Comparing against Baseline

In [104]:
vector_index_2021 = VectorStoreIndex(nodes_2021)
vector_query_engine_2021 = vector_index_2021.as_query_engine(
    similarity_top_k=2
)
vector_index_2020 = VectorStoreIndex(nodes_2020)
vector_query_engine_2020 = vector_index_2020.as_query_engine(
    similarity_top_k=2
)
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine_2021,
        metadata=ToolMetadata(
            name="tesla_2021_10k",
            description=(
                "Provides information about Tesla financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=vector_query_engine_2020,
        metadata=ToolMetadata(
            name="tesla_2020_10k",
            description=(
                "Provides information about Tesla financials for year 2020"
            ),
        ),
    ),
]

base_sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

In [ ]:
response = base_sub_query_engine.query(
    "Can you compare and contrast the cash flow in 2021 with 2020?"
)
print(str(response))

Generated 2 sub questions.
[tesla_2021_10k] Q: What was the cash flow of Tesla in 2021?
[tesla_2020_10k] Q: What was the cash flow of Tesla in 2020?
[tesla_2020_10k] A: Tesla had a cash flow of $5.94 billion in 2020.
[tesla_2021_10k] A: The cash flow of Tesla in 2021 cannot be determined based on the given context information.
I'm sorry, but the cash flow of Tesla in 2021 is not specified, so a comparison with the 2020 cash flow of $5.94 billion cannot be made.
